In [1]:
import pandas as pd
def get_obs_item(current_obs_name, obs_patient,is_find_history,admid):
# Purpose of the function: retrieves the value of a specific observation for a patient, considering both current and historical admission data if necessary
# current_obs_name: The name of the observation to search for；；；obs_patient: A DataFrame containing all observations for a patient；；；is_find_history: A boolean flag indicating whether to search historical data if no current admission data is found.
# set up function Definition:
    '''
    :param obs_name:
    :param tmp_obs:current Admission obs 
    :param tmp_obs_patient: all Admissions obs for the patient
    :param is_find_history: boolean, if there is no information about the current admission, whether to search from the historical information
    :return: obs value
    '''
# Retrieve Current Admission Observations:Filters the obs_patient DataFrame to get the observations for the current admission (admid).
    current_adm_obs=obs_patient[obs_patient['ADMID']==admid]
# Filter for Specific Observation Type:
    tmpdf = current_adm_obs[current_adm_obs['OBS_TYPE'] == current_obs_name] # last admission
    res = ''
# Check for Empty Result and Historical Search:
  # -If no observations of the specified type are found in the current admission (len(tmpdf) == 0):
  # -Check if historical data should be considered (is_find_history).
  # -Filter the obs_patient DataFrame for the specified observation type (current_obs_name).
  # -If historical data exists, retrieve the first recorded value.
# If observations are found, retrieve the first recorded value from the current admission.
# print(obs_name,tmp_obs,tmp_obs_patient,is_find_history)
    if len(tmpdf) == 0:
        if is_find_history:
            tmpdf_history = obs_patient[obs_patient['OBS_TYPE'] == current_obs_name]
            if len(tmpdf_history) > 0:
                res = tmpdf_history['REC'].values[0]
    else:
        res = tmpdf['REC'].values[0]
    return res



#The function searches for a specific medical condition (symptom or comorbidity) in a patient's record. If found, it returns the condition's name, its SNOMED CT code, and the date it was noted. If not found, it returns empty strings.
#The function expects SNOMED CT codes to be used for identifying medical conditions.
#item_name: The name of the symptom/comorbidity to look for；；；tmp_symptoms: A DataFrame containing the patient's symptoms/comorbidities；；；sympotoms_snomed_info: A dictionary containing symptom/comorbidity IDs.
#Filters the tmp_symptoms DataFrame to include only rows where the PROBLEM column contains IDs matching those in the sympotoms_snomed_info dictionary for the specified item_name.
#Checks if the filtered DataFrame tmp_df has any records.
  #-If Records Found:Concatenates the PROB_NAME and PROBLEM values from the first row of tmp_df, and returns this along with the NOTED_DATE.
  #-If No Records Found:Returns two empty strings, indicating no matching records found.
def get_symptoms_comorbidities_item(item_name,tmp_symptoms,sympotoms_snomed_info):

    tmp_df=tmp_symptoms[tmp_symptoms['PROBLEM'].isin(sympotoms_snomed_info[item_name+'_ID'])]
    if len(tmp_df)>0:
        return tmp_df['PROB_NAME'].values[0]+'['+str(tmp_df['PROBLEM'].values[0])+']',tmp_df['NOTED_DATE'].values[0]
    else:
        return '',''



#This function efficiently retrieves unique admission IDs for patients with specified medical conditions, ensuring both current and historical data are considered when extracting these IDs.
#The get_admins function identifies and retrieves admission IDs for patients with specific medical conditions, based on a list of SNOMED IDs. It returns a concatenated string of unique admission IDs and the count of these admissions.
#Defines the function get_admins which takes two arguments:tmp_data: A DataFrame containing patient data；；；snomed_info: A list of SNOMED IDs representing specific medical conditions.
#Filters the tmp_data DataFrame to include only rows where the PROBLEM column matches any of the SNOMED IDs in snomed_info.
#Check for Records: If the filtered DataFrame tmp_df is not empty (len(tmp_df) > 0):
  #-Extract unique admission IDs (ADMID) from tmp_df.
  #-Convert these admission IDs to a set (to ensure uniqueness), then to a list.
  #-Join the list of unique admission IDs into a single string, separated by |.
  #-Return this concatenated string and the count of unique admission IDs.
#If No Records Found: If tmp_df is empty, return an empty string and zero, indicating no matching admissions.
def get_admins(tmp_data,snomed_info):

    tmp_df=tmp_data[tmp_data['PROBLEM'].isin(snomed_info)]
    if len(tmp_df)>0:
        admid=list(set(tmp_df['ADMID'].values.tolist()))

        return '|'.join(admid),len(admid)
    else:
        return '',0



#The function processes ITU admission data for specified admission IDs, calculates the total number of ICU (Intensive Care Unit) admissions and the total duration (in hours) spent in the ICU for a set of admission IDs.
#Initializes counters to track the number of ICU admissions (num_itu) and the total hours spent in the ICU (hour_itu).
#Filters the data_ITU DataFrame to include only rows where the ADMID matches the current item (admission ID).
#Checks if there are any ICU admissions for the current admission ID. If there are, increments the num_itu counter by the number of rows in tmp_df.
#Loop through each ICU admission record: Iterates over each row in the filtered tmp_df DataFrame.
#Extract Start and End Dates/Times: Retrieves the start date (start_date), start time (start_time), end date (end_date), and end time (end_time) from the row.
#Calculate ICU Duration:
  #-If END_TIME is valid, calculates the duration in hours by:
  #-Computing the time difference between end_time and start_time in seconds and converting to hours.
#Adding the difference in days between end_date and start_date multiplied by 24 to get the total hours spent in the ICU for that admission.
#Returns the total number of ICU admissions (num_itu) and the total duration spent in the ICU (hour_itu).
def get_itu_time(data_ITU,admids):
    num_itu=0
    hour_itu=0
    for item in admids:
        tmp_df=data_ITU[data_ITU['ADMID']==item]
        if len(tmp_df)>0:
            num_itu+=len(tmp_df)
            for index,row in tmp_df.iterrows():
                start_date=row['START_DATE']
                start_time=pd.to_datetime(row['START_TIME'], format='%H:%M:%S')
                end_date=row['END_DATE']
                if len(str(row['END_TIME']))>4:
                    end_time=pd.to_datetime(row['END_TIME'], format='%H:%M:%S')
                    hour_itu+= (end_time - start_time).seconds/3600+(end_date-start_date)*24
    return num_itu,hour_itu


#The get_surg_item function retrieves information about a specific surgical procedure for a patient from their surgical history. It checks if the procedure code is present in the patient's records, and if found, returns the name and date of the procedure.
#Filters the tmp_surg DataFrame to include only rows where the PROC_CODE column matches any of the procedure codes in the surg_info dictionary for the specified item_name.
#Check for Records: If the filtered DataFrame tmp_df is not empty (len(tmp_df) > 0):
  #-Retrieves the surgical history name (SURG_HX_NAME) and procedure code (PROC_CODE) from the first row of tmp_df.
  #-Concatenates the name and procedure code into a single string.
  #-Retrieves the date of the procedure (HX_DATE) from the first row.
  #-Returns the concatenated string and the procedure date.
#If No Records Found: If tmp_df is empty, returns two empty strings, indicating no matching records found.
def get_surg_item(item_name,tmp_surg,surg_info):
    tmp_df=tmp_surg[tmp_surg['PROC_CODE'].isin(surg_info[item_name+'_ID'])]
    if len(tmp_df)>0:
        return tmp_df['SURG_HX_NAME'].values[0]+'['+str(tmp_df['PROC_CODE'].values[0])+']',tmp_df['HX_DATE'].values[0]
    else:
        return '',''


#### the function below is defined during coding experiment process, not to be used for dissertation submission
#def get_medication_item(item_name,tmp_surg,surg_info):
   # tmp_df=tmp_surg[tmp_surg['PROC_CODE'].isin(surg_info[item_name+'_ID'])]
   # if len(tmp_df)>0:
   #     return tmp_df['SURG_HX_NAME'].values[0]+'['+str(tmp_df['PROC_CODE'].values[0])+']',tmp_df['HX_DATE'].values[0]
   # else:
    #    return '',''


#### the function below is defined during coding experiment process, not to be used for dissertation submission
#def get_medical_history_item(tmp_dataframe, all_snomed_names):
   #'''
   # :param item_name: characteristics 
   # :param tmp_dataframe: data
   # :param all_snomed_names: snomed CT name
   # :return:
   # '''
   # tmp_df = tmp_dataframe[tmp_dataframe['PROBLEM'].isin(all_snomed_names)]
   # if len(tmp_df) > 0:
   #     return tmp_df['MED_HX_NAME'].values[0], tmp_df['HX_DATE'].values[0]
   # else:
   #     return '', ''

### 1 Load EHR dataset as provided in .txt format

In [2]:
#-- coding:utf-8 --
###########################
# Main Program Starts Here
###########################


import pandas as pd
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# load .txt data files; ensure character set compatibility for medical data 
data_Adm= pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Adm.txt',sep='\t')
data_Algy = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Algy.txt',sep='\t')
data_Cancer= pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Cancer.txt',sep='\t')
data_Demog= pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Demog.txt',sep='\t')
data_ITU = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/ITU.txt',sep='\t')
data_MedHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/MedHX.txt',sep='\t',encoding='iso-8859-1',dtype={'PROBLEM':int})
data_Meds = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Meds.txt',sep='\t',encoding='iso-8859-1')
data_Obs= pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Obs.txt',sep='\t')
data_Probs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Probs.txt',sep='\t',encoding='iso-8859-1',dtype={'PROBLEM':int})
data_SurgHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/SurgHX.txt',sep='\t')
data_Thtr = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Thtr.txt',sep='\t')


In [4]:
data_Obs

,ADMID,PATIENTID,REC_DATE,REC_TIME,AGE_GRP_AT_REC,OBS_TYPE,REC
0,1BB1B77784EC1C7D628E2299BEAE954DC848CFC75BE171...,DCFB120405304487EB31051A874ACE2DFDB35EDD5B5348...,31052.0,20:27:00,75 - 79,HR,67.0
1,DFC93B4648E6EA332394C1D60413CD9C419EC943EFA961...,A756FD7554C1F5CC32A5F6775D1380518D3A4857A3AB41...,27753.0,10:00:00,70 - 74,HR,71.0
2,7032F1DA89F23956923EE2688685C7B8BFBC0480CB57B6...,9D172CC26EB507C1755CA1CFB0210133965F4126AF9E9E...,33065.0,21:51:00,85 - 89,HR,67.0
3,FAC02D171AD93398C158A834A4EEEF82C9BB2E3EA5194C...,7D8DE7AD712817DAC37C4986E5DB5A33A08C8BCD6041B3...,24443.0,14:00:00,60 - 64,HR,98.0
4,49EE842DDB72E494EFB881615BA21B4BEA4F68ED92DA9A...,A222451BA73473DC2CEC28A4BE27B39581798ACAD3119E...,20745.0,14:03:00,50 - 54,HR,89.0
...,...,...,...,...,...,...,...
6716277,D84625EDE8692CD4E027F2F7850341E9DA6A5A38E1CF45...,29AE21C9CAFFE2EB7EE5433B233ABCBBDB0279F542D3D1...,25358.0,20:16:00,60 - 64,RR,22.0
6716278,71EDFD574AD10F39E4F3A110A95760187C83A23A76ECDE...,46B4DC5EAF3E05C753965EE14B7B25963E7035BD3E4C57...,38402.0,05:47:00,95 - 99,RR,19.0
6716279,439E1B1C8BB83F7CF6EEDB21CB87AC09CD19A7C74FA117...,FF3AC2F1FC853F775C1314C9C42F8D54B1AAC8BD167CE1...,36782.0,21:46:00,95 - 99,O2SAT,96.0
6716280,31C09000FE864304D6965F63B69ED19E14E8653A301690...,3EEB07DDFCFDBD7E0B10976B8592D3AB5F4A0EFC2061FF...,33145.0,01:00:00,85 - 89,O2SAT,89.0


###  2 Fing cardiovascular patients, merge Demography and Cancer attributes

In [3]:
# 1. merge demog and cancer Datasets; left join cancer data on 'PATIENTID'.
data_all=pd.merge(data_Demog,data_Cancer,on='PATIENTID',how='left')

# 2. lReads two sheets from an Excel file containing SNOMED codes. snomed_ct_all contains a reference table, and snomed_ct_detail provides detailed SNOMED codes.
snomed_ct_all=pd.read_excel('/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/Python/page snomed code.xlsx',sheet_name='Reference Table')
snomed_ct_detail=pd.read_excel('/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/Python/page snomed code.xlsx',sheet_name='Detailed Snomed codes')

# 3. Identify Cardiovascular Conditions:
#Filters the reference table to get names of sheets related to cardiovascular diseases that are marked as used. 
#Iterates through these sheets to collect the corresponding SNOMED IDs from the detailed codes sheet.
cardiovascular_sheets=snomed_ct_all[(snomed_ct_all['type']=='Cardiovascular Disease')&(snomed_ct_all['Used']==1)]['name'].values
cardiovascular_snomed_ids=[]
for item in cardiovascular_sheets:
    snomed_ct_now=snomed_ct_detail[snomed_ct_detail['High Level Category']==item]
    cardiovascular_snomed_ids+=snomed_ct_now['Snomed SCTID and ConceptID'].values.tolist()

# 4. Identify COVID Conditions:
#Filters the reference table for COVID-related conditions and collects the corresponding SNOMED IDs from the detailed codes sheet, similar to the cardiovascular process.
covid_sheets=snomed_ct_all[(snomed_ct_all['type']=='medhx')&(snomed_ct_all['Used']==1)]['name'].values
covid_snomed_ids=[]
for item in covid_sheets:
    snomed_ct_now=snomed_ct_detail[snomed_ct_detail['High Level Category']==item]
    covid_snomed_ids+=snomed_ct_now['Snomed SCTID and ConceptID'].values.tolist()

# 5. Annotate Patient Data:
#Iterates through each patient, filters the problems DataFrame (data_Probs) to get the patient's records.
#Uses the get_admins function to find cardiovascular and COVID-related admissions.
#Annotates the patient data with admission IDs and counts for cardiovascular and COVID conditions.
for index,row in tqdm(data_all.iterrows()):
    pid=row['PATIENTID']
    filtered_df = data_Probs[data_Probs['PATIENTID']==pid]
    current_cardiovascular_adminssions,current_num=get_admins(filtered_df,cardiovascular_snomed_ids)
    data_all.loc[index, 'cardiovascular_adminID']=current_cardiovascular_adminssions
    data_all.loc[index, 'cardiovascular_adminID_num']=current_num
    current_covid_adminssions,current_num=get_admins(filtered_df,covid_snomed_ids)
    data_all.loc[index, 'covid_adminID']=current_covid_adminssions
    data_all.loc[index, 'covid_adminID_num']=current_num

#Filters the dataset to include only patients with at least one cardiovascular admission and less than two COVID-19 admissions.
data_all=data_all[(data_all['cardiovascular_adminID_num']>0)&(data_all['covid_adminID_num']<2)]    # at least 1 cvd and 1covid admi  

data_all


7500it [00:27, 275.94it/s]


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,covid_adminID_num
2,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,0.0
4,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,0.0
5,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,1.0
6,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,0.0
10,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7490,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,0.0
7492,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,0.0
7494,FA124AD149310859E6A9B04FD8AF4ABFE7636E65927C7B...,1,NG22,A,27281.0,65 - 69,Y,815A9943A50942B310899B74C076EF07651ED078EB2B2B...,1.0,,0.0
7497,D3B259F18829886B92585D8E9BE93106B1AC362CAAE17F...,2,CB24,A,38680.0,100 - 104,Y,00E9929E6093E25544CD29E75684278C69BB30CCA1FBC7...,3.0,,0.0


## 3 Admission info processing to count for CVD 14 snomoeds

In [4]:
# Iterates over patient data to determine if patients are deceased, identify relevant admission IDs for cardiovascular or COVID conditions, and merge additional admission information into the dataset.

# 1.Uses a loop to iterate over each row in the data_all DataFrame, extracting patient ID (pid), cardiovascular admission IDs (cardiovascular_adminID), and COVID admission ID (covid_adminID).
for index,row in tqdm(data_all.iterrows()):
    pid=row['PATIENTID']
    cardiovascular_adminID=row['cardiovascular_adminID'].split('|')
    covid_adminID=row['covid_adminID']
    
    
# 2.Check if Patient is Deceased:Checks if the DOD (date of death) is a valid number. If so, marks the patient as deceased (DEAD = 1); otherwise, marks as not deceased (DEAD = 0).
    if str(row['DOD']).replace('.','').isdigit():
        data_all.loc[index,'DEAD']=1
    else:
        data_all.loc[index,'DEAD']=0
  
# 3.Identify COVID Admissions:   
# Check COVID Admission: If covid_adminID is in the list of cardiovascular_adminIDs:
  #-Marks the patient as having a COVID admission (covid = 1).
  #-Sets the adminID to the covid_adminID.
#If No COVID Admission: If covid_adminID is not in the list:
  #-Marks the patient as not having a COVID admission (covid = 0).
  #-Retrieves the admission dates for cardiovascular admissions.
  #-Sorts and selects the latest cardiovascular admission ID.
    if covid_adminID in cardiovascular_adminID:
        data_all.loc[index,'covid']=1
        data_all.loc[index,'adminID']=covid_adminID
    else:
        data_all.loc[index,'covid']=0
        admin_dates=[]
        for itemid in cardiovascular_adminID:
            tmp=data_Adm[data_Adm['ADMID']==itemid]
            admin_dates.append(tmp['ADM_DATE'].values[0])
        ind=np.argsort(admin_dates)
        data_all.loc[index,'adminID']=cardiovascular_adminID[ind[-1]]

# 4.Merge Additional Admission Information:merge problem,admission info 
# merges additional admission details from the data_Adm DataFrame into data_all based on the adminID and ADMID columns. Drops the duplicate ADMID column after merging.
data_all=pd.merge(data_all,data_Adm[['ADMID',	'ADM_DATE'	,	'AGE_GRP_AT_ADM'	,'DIS_DATE'	,		'DISCH_DEST',	'TFC_CODE','TFC_DESC'	,'ADM_SOURCE'	,'NAT_ADM_METH']],left_on='adminID',right_on='ADMID',how='left')#join covid patient    
data_all = data_all.drop('ADMID', axis=1)


# 5.Print Number of Patients and Display Data:
#Prints the number of patients in the data_all DataFrame and displays the DataFrame.
print('3. num of patient: ',len(data_all))
data_all

2493it [00:03, 693.07it/s]

3. num of patient:  2493


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,covid,adminID,ADM_DATE,AGE_GRP_AT_ADM,DIS_DATE,DISCH_DEST,TFC_CODE,TFC_DESC,ADM_SOURCE,NAT_ADM_METH
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,0.0,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,34565,85 - 89,34600,79,328,STROKE MEDICINE,19,21
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,0.0,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,29275,70 - 74,29287,19,107,VASCULAR SURGERY,51,2B
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,1.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,36435,90 - 94,36473,79,430,GERIATRIC MEDICINE,19,21
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,0.0,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,27802,70 - 74,27810,51,328,STROKE MEDICINE,19,21
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,0.0,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,29143,70 - 74,29155,79,300,GENERAL MEDICINE,51,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,0.0,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,29070,70 - 74,29081,19,110,TRAUMA & ORTHOPAEDICS,19,11
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,...,0.0,94A29E441BFDF53DD565EA0F396BE48BFF3FC3C562EBF4...,27947,70 - 74,27961,19,340,RESPIRATORY MEDICINE,19,24
2490,FA124AD149310859E6A9B04FD8AF4ABFE7636E65927C7B...,1,NG22,A,27281.0,65 - 69,Y,815A9943A50942B310899B74C076EF07651ED078EB2B2B...,1.0,,...,0.0,815A9943A50942B310899B74C076EF07651ED078EB2B2B...,27238,65 - 69,27261,19,320,CARDIOLOGY,19,24
2491,D3B259F18829886B92585D8E9BE93106B1AC362CAAE17F...,2,CB24,A,38680.0,100 - 104,Y,00E9929E6093E25544CD29E75684278C69BB30CCA1FBC7...,3.0,,...,0.0,00E9929E6093E25544CD29E75684278C69BB30CCA1FBC7...,37673,95 - 99,37690,85,430,GERIATRIC MEDICINE,19,21


In [10]:
data_all

,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,covid,adminID,ADM_DATE,AGE_GRP_AT_ADM,DIS_DATE,DISCH_DEST,TFC_CODE,TFC_DESC,ADM_SOURCE,NAT_ADM_METH
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,0.0,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,34565,85 - 89,34600,79,328,STROKE MEDICINE,19,21
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,0.0,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,29275,70 - 74,29287,19,107,VASCULAR SURGERY,51,2B
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,1.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,36435,90 - 94,36473,79,430,GERIATRIC MEDICINE,19,21
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,0.0,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,27802,70 - 74,27810,51,328,STROKE MEDICINE,19,21
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,0.0,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,29143,70 - 74,29155,79,300,GENERAL MEDICINE,51,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,0.0,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,29070,70 - 74,29081,19,110,TRAUMA & ORTHOPAEDICS,19,11
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,654684E4DE8DBF6D5FFF764189B89090232409E58689E1...,7.0,,...,0.0,94A29E441BFDF53DD565EA0F396BE48BFF3FC3C562EBF4...,27947,70 - 74,27961,19,340,RESPIRATORY MEDICINE,19,24
2490,FA124AD149310859E6A9B04FD8AF4ABFE7636E65927C7B...,1,NG22,A,27281.0,65 - 69,Y,815A9943A50942B310899B74C076EF07651ED078EB2B2B...,1.0,,...,0.0,815A9943A50942B310899B74C076EF07651ED078EB2B2B...,27238,65 - 69,27261,19,320,CARDIOLOGY,19,24
2491,D3B259F18829886B92585D8E9BE93106B1AC362CAAE17F...,2,CB24,A,38680.0,100 - 104,Y,0D6F9A5C98CF2B9FAFFE38288E1E6DFFC636B663DE233B...,3.0,,...,0.0,00E9929E6093E25544CD29E75684278C69BB30CCA1FBC7...,37673,95 - 99,37690,85,430,GERIATRIC MEDICINE,19,21


## 4 Get Obs info and find in-hospitaldeath

In [5]:
# This code efficiently processes patient data to determine in-hospital deaths, retrieve relevant observations, and update the dataset accordingly. 
# It ensures that only in-hospital deaths are considered in the final dataset.
# Get info for every patient - array run once per patient 

# 1.Initialize Observation Lists:creates two lists:
  #-obs_name_list: Names of observations to retrieve.
  #-is_find_history: Indicates whether to search historical data for each observation.
obs_name_list=['WEIGHT','HEIGHT','DBP','SBP','RR','HR','O2SAT']
is_find_history=[True,True,False,False,False,False,False]

# 2.Iterate through Patient Data:
#Uses a loop to iterate over each row in the data_all DataFrame, extracting the admission ID (admid) and patient ID (pid).
for index,row in tqdm(data_all.iterrows()):
    admid=row['adminID']
    pid=row['PATIENTID']

# 3.Determine In-Hospital Death:
# Checks if the patient is deceased and whether the date of death (DOD) falls within the admission (ADM_DATE to DIS_DATE). If true, marks the patient as having died in the hospital (In-hospital death = 1); otherwise, marks as 0.
    if row['DEAD']==1 and row['DOD']>=row['ADM_DATE'] and row['DOD']<=row['DIS_DATE']:
        data_all.loc[index, 'In-hospital death']=1
    else:
        data_all.loc[index, 'In-hospital death']=0

# 4.Retrieve Observations for Each Patient:   
  #Filters data_Obs to get all observations for the current patient (current_obs_patient).
  #Iterates through each observation type and its corresponding history flag.
  #Calls get_obs_item to retrieve the observation value.
  #Updates the data_all DataFrame with the retrieved observation value if it’s not empty.
    #current_obs=data_Obs[data_Obs['ADMID']==admid].sort_values(by='REC_DATE')
    current_obs_patient=data_Obs[data_Obs['PATIENTID']==pid]
    for item,find_history in zip(obs_name_list,is_find_history):
        current_res=get_obs_item(item,current_obs_patient,find_history,admid)
        if current_res !='':
           data_all.loc[index, item]=current_res

# 5.Remove out of hospital dead patients
data_all=data_all[(data_all['DEAD']==0)|((data_all['DEAD']==1)&(data_all['In-hospital death']==1))]
data_all    

2493it [06:44,  6.17it/s]


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,ADM_SOURCE,NAT_ADM_METH,In-hospital death,WEIGHT,HEIGHT,DBP,SBP,RR,HR,O2SAT
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,19,21,1.0,74.9992,74.764,64.0,123.0,18.0,64.0,97.0
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,51,2B,0.0,78.9991,75.000,75.0,129.0,15.0,108.0,95.0
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,19,21,1.0,44.3996,65.984,66.0,95.0,17.0,78.0,97.0
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,19,21,0.0,74.9992,70.866,73.0,210.0,18.0,88.0,98.0
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,51,2B,1.0,99.9990,69.291,51.0,131.0,22.0,59.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2B2DC2C477620D9F109B76DF0FBF89D3EBA71169EDA6B9...,2,CB4,A,25195.0,60 - 64,Y,A0314EDE75FA1C4616D579C172FC358CB0E304699796A4...,7.0,A31B4EA3D308DE7B645D2DEC7629BA904B86AA84C8B3E4...,...,19,21,1.0,134.4986,66.000,65.0,212.0,18.0,58.0,97.0
2487,07D96DC81E62CE5427EC0E569D87CFF3BC1CE9C67AD5D2...,1,CB24,A,21296.0,50 - 54,N,75296F309875068EB8E67D84C0D07096E9B15CBF1069DD...,1.0,,...,19,21,1.0,104.9990,68.110,61.0,109.0,27.0,96.0,93.0
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,19,11,0.0,111.1987,74.094,67.0,109.0,17.0,95.0,97.0
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,...,19,24,0.0,72.3491,66.614,76.0,115.0,18.0,66.0,93.0


5 Get symptoms, Cardiovascular Disease, BMI and comorbidities, add into all data table

In [7]:
# This code enriches patient records with detailed information about symptoms and comorbidities by leveraging SNOMED CT codes and ensuring each patient’s record is complete with the relevant clinical data. 

# 1. Read SNOMED Codes from Excel Files:
snomed_ct_all=pd.read_excel('/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/Python/page snomed code.xlsx',sheet_name='Reference Table')
snomed_ct_detail=pd.read_excel('/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/Python/page snomed code.xlsx',sheet_name='Detailed Snomed codes')

# 2. Identify Relevant Symptom and Comorbidity Sheets:
# Filters snomed_ct_all to include only rows where the 'type' column contains 'Symptoms', 'Comorbidities', 'Cardiovascular Disease', or 'BMI', and where the 'Used' column is 1. 
# Extracts the 'name' column values into sympotom_sheets
sympotom_sheets=snomed_ct_all[(snomed_ct_all['type'].str.contains('Symptoms|Comorbidities|Cardiovascular Disease|BMI'))&(snomed_ct_all['Used']==1)]['name'].values


# 3. Create Symptom SNOMED Info Dictionary:
# Initializes an empty dictionary sympotoms_snomed_info. 
# Iterates over each item in sympotom_sheets, filters snomed_ct_detail to get rows for the current item, and populates the dictionary with SNOMED IDs.
sympotoms_snomed_info={ }
for item in sympotom_sheets:
    snomed_ct_now=snomed_ct_detail[snomed_ct_detail['High Level Category']==item]
    sympotoms_snomed_info[item+'_ID']=snomed_ct_now['Snomed SCTID and ConceptID'].values
    # sympotoms_snomed_info[item+'_name']=snomed_ct_now['Code description'].values

# 4. Iterate through Patient Data:
# Uses a loop to iterate over each row in data_all, extracting admission IDs (admids) and patient ID (pid). 
# Filters data_Probs to get problem records for the current admissions, sorted by NOTED_DATE
for index,row in tqdm(data_all.iterrows()):
    admids=row['adminID'].split('|')
    pid=row['PATIENTID']
    filtered_df = data_Probs[data_Probs['ADMID'].isin(admids)].sort_values(by='NOTED_DATE')

# 5. Retrieve and Annotate Symptom/Comorbidity Data:
# Iterates over each symptom/comorbidity sheet (sympotom_sheets), calls get_symptoms_comorbidities_item to retrieve the symptom/comorbidity and its date. 
# If a symptom is found, updates data_all with the symptom name and date.
    for item in sympotom_sheets:
        current_symptom,sympotoms_date=get_symptoms_comorbidities_item(item,filtered_df,sympotoms_snomed_info)
        if current_symptom !='':
            data_all.loc[index, item]=current_symptom
            data_all.loc[index, item+'_date']=sympotoms_date
    
# 6. Return the Updated Data:
data_all    

1730it [00:05, 292.78it/s]


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,Cough,Cough_date,Vomiting,Vomiting_date,Nausea,Nausea_date,Cancer,Cancer_date,Headache,Headache_date
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2B2DC2C477620D9F109B76DF0FBF89D3EBA71169EDA6B9...,2,CB4,A,25195.0,60 - 64,Y,A0314EDE75FA1C4616D579C172FC358CB0E304699796A4...,7.0,A31B4EA3D308DE7B645D2DEC7629BA904B86AA84C8B3E4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,07D96DC81E62CE5427EC0E569D87CFF3BC1CE9C67AD5D2...,1,CB24,A,21296.0,50 - 54,N,75296F309875068EB8E67D84C0D07096E9B15CBF1069DD...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 6 ITU info

In [8]:
#This code processes patient data to calculate and update the number of ICU admissions and the total time spent in the ICU for each patient 
#Ensuring the dataset is enriched with detailed ICU information.

#1. Uses a loop to iterate over each row in the data_all DataFrame. 
#The tqdm function provides a progress bar to monitor the iteration progress.
#2. Extracts the admission IDs (admids) by splitting the adminID string using the '|' delimiter. Also extracts the patient ID (pid).
#3. Calls the get_itu_time function, passing in the ICU data (data_ITU) and the list of admission IDs (admids). 
#This function returns the number of ICU admissions (num_itu) and total hours spent in the ICU (hour_itu) for the given admissions.
#4. Updates the data_all DataFrame at the current index with the number of ICU admissions (ITU_num) and total hours spent in the ICU (ITU_hour).

for index,row in tqdm(data_all.iterrows()):
    admids=row['adminID'].split('|')
    pid=row['PATIENTID']
    num_itu,hour_itu=get_itu_time(data_ITU,admids)
    data_all.loc[index, 'ITU_num']=num_itu
    data_all.loc[index, 'ITU_hour']=hour_itu
        
data_all

1730it [00:01, 1218.34it/s]


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,Vomiting,Vomiting_date,Nausea,Nausea_date,Cancer,Cancer_date,Headache,Headache_date,ITU_num,ITU_hour
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2B2DC2C477620D9F109B76DF0FBF89D3EBA71169EDA6B9...,2,CB4,A,25195.0,60 - 64,Y,A0314EDE75FA1C4616D579C172FC358CB0E304699796A4...,7.0,A31B4EA3D308DE7B645D2DEC7629BA904B86AA84C8B3E4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2487,07D96DC81E62CE5427EC0E569D87CFF3BC1CE9C67AD5D2...,1,CB24,A,21296.0,50 - 54,N,75296F309875068EB8E67D84C0D07096E9B15CBF1069DD...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


## 7 Add Surgery info

In [11]:
# This code enriches patient records with detailed surgical information by leveraging SNOMED CT codes and ensures that the dataset is updated and saved with the most relevant surgical data for further analysis. 

# 1.Initialize an Empty Dictionary for Surgery SNOMED Info:
# Creates an empty dictionary to store surgical SNOMED CT codes.
# Filters the SNOMED CT reference table for surgery types that are marked as used.
surg_snomed_info={}
surg_sheets=snomed_ct_all[(snomed_ct_all['type']=='surgery')&(snomed_ct_all['Used']==1)]['name'].values

# 2.Populating Surgical SNOMED CT Information:
# For each surgery type, retrieves the corresponding SNOMED CT codes from the detailed table.
# Stores these codes in the surg_snomed_info dictionary.
for item in surg_sheets:
    snomed_ct_now=snomed_ct_detail[snomed_ct_detail['High Level Category']==item]
    surg_snomed_info[item+'_ID']=snomed_ct_now['Snomed SCTID and ConceptID'].values

# 3.Processing Patient Data:
# iterates through each row in the main dataset (data_all).
# Uses tqdm for a progress bar.
for index,row in tqdm(data_all.iterrows()):
   
# 4.Extracting Patient Information:
    admids=row['adminID'].split('|')
    pid=row['PATIENTID']
    
# 5.Filtering Surgical History, processing Each Surgery Type:
#Filters the surgical history data for the current patient's admissions.
#For each surgery type, calls get_surg_item function to retrieve surgery information.
#If a surgery is found, updates the main dataset with the surgery name and date.
    filtered_df = data_SurgHX[data_SurgHX['ADMID'].isin(admids)]
    for item in surg_sheets:
        current_surg,surg_date=get_surg_item(item,filtered_df,surg_snomed_info)
        if current_surg !='':
            data_all.loc[index, item]=current_surg
            data_all.loc[index, item+'_date']=surg_date

# 6.Saving and Displaying Results:
data_all.to_excel('/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/30Oct_final/data_all.xlsx',index=False)
data_all
    

1730it [00:03, 490.24it/s]


,PATIENTID,GENDER,PCODE_AREA,ETHNICITY,DOD,AGE_GRP_AT_DEATH,CANCER,cardiovascular_adminID,cardiovascular_adminID_num,covid_adminID,...,Vomiting,Vomiting_date,Nausea,Nausea_date,Cancer,Cancer_date,Headache,Headache_date,ITU_num,ITU_hour
0,D1917A064ADE19D601DF43DF4AC0EDB40C8B2E143CD06A...,1,CB10,A,34600.0,85 - 89,Y,3F43772988AD82E2D2FD50E9C3EB251DA4E8253B564E25...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,A51387C9064B83E379CF58B48B1D6935262F7B950C51D5...,1,PE2,A,NaN,NaN,N,E4AA425EC2879B4FD021A18840F774D29A9B0D916AC932...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,6CD89B9487B66816F10BA35B5C398605DF0B9BFD5C2FA0...,1,CB22,A,36472.0,90 - 94,Y,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,4.0,EB9794A6AD9DCEA69A8F8D18161BAE4A474F07C5C2DE47...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,0A91111CB6E0008C8519E366DADFB95CD21249080280E6...,1,PE28,A,NaN,NaN,N,58DA88A9BA349E014B4C8B32EFB607F87FFF34F286D40A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,5ED4DB0EF402CFA869F9B5C8779EDCE193982E9D2B87AD...,1,CM16,NaN,29155.0,70 - 74,N,E5874F09AD7E5302BEB0754862C2D956FC52E07BE2849A...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2B2DC2C477620D9F109B76DF0FBF89D3EBA71169EDA6B9...,2,CB4,A,25195.0,60 - 64,Y,A0314EDE75FA1C4616D579C172FC358CB0E304699796A4...,7.0,A31B4EA3D308DE7B645D2DEC7629BA904B86AA84C8B3E4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2487,07D96DC81E62CE5427EC0E569D87CFF3BC1CE9C67AD5D2...,1,CB24,A,21296.0,50 - 54,N,75296F309875068EB8E67D84C0D07096E9B15CBF1069DD...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2488,9555EF433E379D50AA57DBEE581ED61B1012274C90B3B7...,1,SG8,A,NaN,NaN,Y,8EB068666E188DE523A1BE74E995D31E17181DD1041B13...,1.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2489,597228D385F35DDA9F26685D61BC7DA28F08F749F275A5...,1,CB1,C,NaN,NaN,Y,6832D1C85BBEA7A3B824E62DE277E31B1B4BE08C340138...,7.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [12]:
import pandas as pd

# Load the dataset
file_path = "/Users/r.h.wang/Documents/Mst HD/0. Disso Writing/Analytics/30Oct_final/data_all.xlsx"
data = pd.read_excel(file_path)

# Print column names to verify
print("Column Names in Dataset:")
print(data.columns)


Column Names in Dataset:
Index(['PATIENTID', 'GENDER', 'PCODE_AREA', 'ETHNICITY', 'DOD',
       'AGE_GRP_AT_DEATH', 'CANCER', 'cardiovascular_adminID',
       'cardiovascular_adminID_num', 'covid_adminID', 'covid_adminID_num',
       'DEAD', 'covid', 'adminID', 'ADM_DATE', 'AGE_GRP_AT_ADM', 'DIS_DATE',
       'DISCH_DEST', 'TFC_CODE', 'TFC_DESC', 'ADM_SOURCE', 'NAT_ADM_METH',
       'In-hospital death', 'WEIGHT', 'HEIGHT', 'DBP', 'SBP', 'RR', 'HR',
       'O2SAT', 'Stroke', 'Stroke_date', 'Chronic disease of respiratory ',
       'Chronic disease of respiratory _date', 'COPD', 'COPD_date',
       'Chronic kidney disease', 'Chronic kidney disease_date', 'Hypertension',
       'Hypertension_date', 'Asthma', 'Asthma_date',
       'Diabetes mellitus type 2 ', 'Diabetes mellitus type 2 _date',
       'Chronic liver disease', 'Chronic liver disease_date',
       'Coronary arteriosclerosis ', 'Coronary arteriosclerosis _date',
       'Cardiac arrhythmia', 'Cardiac arrhythmia_date', 'Obesity',

In [14]:
import pandas as pd

# Load the data files
data_Adm = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Adm.txt', sep='\t')
data_Algy = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Algy.txt', sep='\t')
data_Cancer = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Cancer.txt', sep='\t')
data_Demog = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Demog.txt', sep='\t')
data_ITU = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/ITU.txt', sep='\t')
data_MedHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/MedHX.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_Meds = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Meds.txt', sep='\t', encoding='iso-8859-1')
data_Obs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Obs.txt', sep='\t')
data_Probs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Probs.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_SurgHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/SurgHX.txt', sep='\t')
data_Thtr = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Thtr.txt', sep='\t')

# List of datasets
datasets = {
    'data_Adm': data_Adm,
    'data_Algy': data_Algy,
    'data_Cancer': data_Cancer,
    'data_Demog': data_Demog,
    'data_ITU': data_ITU,
    'data_MedHX': data_MedHX,
    'data_Meds': data_Meds,
    'data_Obs': data_Obs,
    'data_Probs': data_Probs,
    'data_SurgHX': data_SurgHX,
    'data_Thtr': data_Thtr
}

# Print table names and their column names
for name, dataset in datasets.items():
    print(f"Table Name: {name}")
    print("Columns:", list(dataset.columns))
    print()


Table Name: data_Adm
Columns: ['ADMID', 'PATIENTID', 'ADM_DATE', 'ADM_TIME', 'AGE_GRP_AT_ADM', 'DIS_DATE', 'DIS_TIME', 'AGE_GRP_AT_DIS', 'DISCH_DEST', 'TFC_CODE', 'TFC_DESC', 'ADM_SOURCE', 'NAT_ADM_METH']

Table Name: data_Algy
Columns: ['ADMID', 'PATIENTID', 'ALGY_DATE', 'AGE_GRP_AT_ALGY', 'ALLERGEN_TYPE', 'ALLERGEN', 'REACTION']

Table Name: data_Cancer
Columns: ['PATIENTID', 'CANCER']

Table Name: data_Demog
Columns: ['PATIENTID', 'GENDER', 'PCODE_AREA', 'ETHNICITY', 'DOD', 'AGE_GRP_AT_DEATH']

Table Name: data_ITU
Columns: ['ADMID', 'PATIENTID', 'START_DATE', 'START_TIME', 'AGE_GRP_AT_START', 'END_DATE', 'END_TIME', 'AGE_GRP_AT_END']

Table Name: data_MedHX
Columns: ['ADMID', 'PATIENTID', 'HX_DATE', 'AGE_GRP_AT_HX_DATE', 'PROBLEM', 'MED_HX_NAME']

Table Name: data_Meds
Columns: ['ADMID', 'PATIENTID', 'MEDICATION', 'MED_ROUTE', 'SCHED_START_DATE', 'SCHED_START_TIME', 'AGE_GRP_AT_SCHED_START', 'SCHED_END_DATE', 'SCHED_END_TIME', 'AGE_GRP_AT_SCHED_END', 'DISCON_DATE', 'DISCON_TIME', '

In [15]:
import pandas as pd

# Load the data files
data_Adm = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Adm.txt', sep='\t')
data_Algy = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Algy.txt', sep='\t')
data_Cancer = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Cancer.txt', sep='\t')
data_Demog = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Demog.txt', sep='\t')
data_ITU = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/ITU.txt', sep='\t')
data_MedHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/MedHX.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_Meds = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Meds.txt', sep='\t', encoding='iso-8859-1')
data_Obs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Obs.txt', sep='\t')
data_Probs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Probs.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_SurgHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/SurgHX.txt', sep='\t')
data_Thtr = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Thtr.txt', sep='\t')

# List of datasets
datasets = {
    'data_Adm': data_Adm,
    'data_Algy': data_Algy,
    'data_Cancer': data_Cancer,
    'data_Demog': data_Demog,
    'data_ITU': data_ITU,
    'data_MedHX': data_MedHX,
    'data_Meds': data_Meds,
    'data_Obs': data_Obs,
    'data_Probs': data_Probs,
    'data_SurgHX': data_SurgHX,
    'data_Thtr': data_Thtr
}

def find_primary_keys(df):
    primary_keys = []
    for col in df.columns:
        if df[col].is_unique:
            primary_keys.append(col)
    return primary_keys

def find_foreign_keys(df, datasets):
    foreign_keys = []
    for col in df.columns:
        for key, dataset in datasets.items():
            if col in dataset.columns and df[col].isin(dataset[col]).all():
                foreign_keys.append((col, key))
    return foreign_keys

# Identify primary keys and foreign keys
for name, dataset in datasets.items():
    print(f"Table Name: {name}")
    primary_keys = find_primary_keys(dataset)
    print("Primary Keys:", primary_keys)
    
    foreign_keys = find_foreign_keys(dataset, datasets)
    print("Foreign Keys:", foreign_keys)
    print()


Table Name: data_Adm
Primary Keys: ['ADMID']
Foreign Keys: [('ADMID', 'data_Adm'), ('PATIENTID', 'data_Adm'), ('PATIENTID', 'data_Cancer'), ('PATIENTID', 'data_Demog'), ('ADM_DATE', 'data_Adm'), ('ADM_TIME', 'data_Adm'), ('AGE_GRP_AT_ADM', 'data_Adm'), ('DIS_DATE', 'data_Adm'), ('DIS_TIME', 'data_Adm'), ('AGE_GRP_AT_DIS', 'data_Adm'), ('DISCH_DEST', 'data_Adm'), ('TFC_CODE', 'data_Adm'), ('TFC_DESC', 'data_Adm'), ('ADM_SOURCE', 'data_Adm'), ('NAT_ADM_METH', 'data_Adm')]

Table Name: data_Algy
Primary Keys: []
Foreign Keys: [('ADMID', 'data_Adm'), ('ADMID', 'data_Algy'), ('PATIENTID', 'data_Adm'), ('PATIENTID', 'data_Algy'), ('PATIENTID', 'data_Cancer'), ('PATIENTID', 'data_Demog'), ('ALGY_DATE', 'data_Algy'), ('AGE_GRP_AT_ALGY', 'data_Algy'), ('ALLERGEN_TYPE', 'data_Algy'), ('ALLERGEN', 'data_Algy'), ('REACTION', 'data_Algy')]

Table Name: data_Cancer
Primary Keys: ['PATIENTID']
Foreign Keys: [('PATIENTID', 'data_Adm'), ('PATIENTID', 'data_Cancer'), ('PATIENTID', 'data_Demog'), ('CANCE

In [16]:
import pandas as pd

# Load the data files
data_Adm = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Adm.txt', sep='\t')
data_Algy = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Algy.txt', sep='\t')
data_Cancer = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Cancer.txt', sep='\t')
data_Demog = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Demog.txt', sep='\t')
data_ITU = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/ITU.txt', sep='\t')
data_MedHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/MedHX.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_Meds = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Meds.txt', sep='\t', encoding='iso-8859-1')
data_Obs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Obs.txt', sep='\t')
data_Probs = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Probs.txt', sep='\t', encoding='iso-8859-1', dtype={'PROBLEM': int})
data_SurgHX = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/SurgHX.txt', sep='\t')
data_Thtr = pd.read_csv('/Users/r.h.wang/Documents/Mst HD/EHR_DataSet/Thtr.txt', sep='\t')

# List of datasets
datasets = {
    'data_Adm': data_Adm,
    'data_Algy': data_Algy,
    'data_Cancer': data_Cancer,
    'data_Demog': data_Demog,
    'data_ITU': data_ITU,
    'data_MedHX': data_MedHX,
    'data_Meds': data_Meds,
    'data_Obs': data_Obs,
    'data_Probs': data_Probs,
    'data_SurgHX': data_SurgHX,
    'data_Thtr': data_Thtr
}

# Function to identify primary keys and potential foreign keys
def identify_keys(datasets):
    primary_keys = {}
    foreign_keys = []

    # Identify primary keys and potential foreign keys
    for name, dataset in datasets.items():
        primary_keys[name] = []
        for col in dataset.columns:
            if dataset[col].is_unique:
                primary_keys[name].append(col)
            if col == 'PATIENTID':
                foreign_keys.append((col, name))

    return primary_keys, foreign_keys

# Identify keys
primary_keys, foreign_keys = identify_keys(datasets)

# Print primary keys and potential foreign keys
print("Primary Keys:")
for name, keys in primary_keys.items():
    print(f"{name}: {keys}")

print("\nPotential Foreign Keys (related to PATIENTID):")
for fk in foreign_keys:
    print(f"Column '{fk[0]}' in table '{fk[1]}'")

# Print data attributes (column names)
print("\nData Attributes (Column Names) by Table:")
for name, dataset in datasets.items():
    print(f"Table '{name}': {list(dataset.columns)}")


Primary Keys:
data_Adm: ['ADMID']
data_Algy: []
data_Cancer: ['PATIENTID']
data_Demog: ['PATIENTID']
data_ITU: []
data_MedHX: []
data_Meds: []
data_Obs: []
data_Probs: []
data_SurgHX: []
data_Thtr: []

Potential Foreign Keys (related to PATIENTID):
Column 'PATIENTID' in table 'data_Adm'
Column 'PATIENTID' in table 'data_Algy'
Column 'PATIENTID' in table 'data_Cancer'
Column 'PATIENTID' in table 'data_Demog'
Column 'PATIENTID' in table 'data_ITU'
Column 'PATIENTID' in table 'data_MedHX'
Column 'PATIENTID' in table 'data_Meds'
Column 'PATIENTID' in table 'data_Obs'
Column 'PATIENTID' in table 'data_Probs'
Column 'PATIENTID' in table 'data_SurgHX'
Column 'PATIENTID' in table 'data_Thtr'

Data Attributes (Column Names) by Table:
Table 'data_Adm': ['ADMID', 'PATIENTID', 'ADM_DATE', 'ADM_TIME', 'AGE_GRP_AT_ADM', 'DIS_DATE', 'DIS_TIME', 'AGE_GRP_AT_DIS', 'DISCH_DEST', 'TFC_CODE', 'TFC_DESC', 'ADM_SOURCE', 'NAT_ADM_METH']
Table 'data_Algy': ['ADMID', 'PATIENTID', 'ALGY_DATE', 'AGE_GRP_AT_ALGY'